In [2]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 18.8 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import DPOTrainer, DPOConfig
import torch

# ============================================
# 1. LOAD DATASET (Same as before)
# ============================================
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf")
print(dataset)
print(dataset["train"][0])

def extract_prompt(example):
    text = example["chosen"]
    parts = text.split("Assistant:")
    prompt = parts[0].strip() if len(parts) > 1 else text.strip()
    return {"prompt": prompt}

dataset = dataset.map(extract_prompt)

def valid(x):
    return all([
        isinstance(x["prompt"], str),
        isinstance(x["chosen"], str),
        isinstance(x["rejected"], str),
        len(x["prompt"]) > 5,
        len(x["chosen"]) > 5,
        len(x["rejected"]) > 5
    ])

dataset = dataset.filter(valid)

print(f"✅ Clean dataset: {len(dataset['train'])} training samples")
print(dataset["train"][0])

# ============================================
# 2. LOAD BASE MODEL & TOKENIZER (Same as before)
# ============================================
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load base models WITHOUT LoRA first
policy_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
ref_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

print("✅ Base models loaded")

# ============================================
# 3. ADD LoRA HERE! (NEW SECTION)
# ============================================
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Configure LoRA
lora_config = LoraConfig(
    r=16,                          # LoRA rank (higher = more parameters, 8-64 typical)
    lora_alpha=32,                 # LoRA scaling factor (usually 2*r)
    target_modules=[
        "c_attn",                  # GPT-2 attention layers
        "c_proj",                  # GPT-2 projection layers
    ],
    lora_dropout=0.05,             # Dropout for LoRA layers
    bias="none",                   # Don't train bias terms
    task_type="CAUSAL_LM"          # Causal language modeling
)

# Apply LoRA to policy model (the one we're training)
policy_model = get_peft_model(policy_model, lora_config)

# Print trainable parameters
policy_model.print_trainable_parameters()
# Output: trainable params: ~0.5M || all params: ~124M || trainable%: 0.4%

print("✅ LoRA applied to policy model")

# Keep reference model frozen (no LoRA)
for p in ref_model.parameters():
    p.requires_grad = False

print("✅ Reference model frozen")

# ============================================
# 4. CONFIGURE TRAINING (Same as before)
# ============================================
training_args = DPOConfig(
    output_dir="./dpo-lora-checkpoints",
    per_device_train_batch_size=4,      # Can increase with LoRA (less memory)
    gradient_accumulation_steps=4,
    learning_rate=5e-5,                 # Can use slightly higher LR with LoRA
    num_train_epochs=3,                 # More epochs with LoRA
    logging_steps=10,
    save_strategy="epoch",
    # evaluation_strategy="steps",
    eval_steps=50,
    report_to="none",
    beta=0.1,
    max_length=512,
    warmup_steps=100,
)

# ============================================
# 5. INITIALIZE DPO TRAINER (Same as before)
# ============================================
from trl import DPOTrainer

trainer = DPOTrainer(
    model=policy_model,                              # LoRA-wrapped model
    ref_model=ref_model,                             # Original frozen model
    processing_class=tokenizer,
    train_dataset=dataset["train"].select(range(5000)),  # More data with LoRA
    eval_dataset=dataset["test"].select(range(500)),
    args=training_args,
)

print("✅ DPO Trainer with LoRA initialized")

# ============================================
# 6. TRAIN (Same as before)
# ============================================
trainer.train()

print("✅ Training complete!")

# ============================================
# 7. SAVE MODEL (DIFFERENT for LoRA!)
# ============================================
# Save LoRA adapters (very small - only ~2MB!)
trainer.save_model("./dpo-lora-checkpoints/final_model")
tokenizer.save_pretrained("./dpo-lora-checkpoints/final_model")

print("✅ LoRA adapters saved!")

# ============================================
# 8. LOAD TRAINED MODEL FOR TESTING (DIFFERENT for LoRA!)
# ============================================
from peft import PeftModel

# Load base model
base_model_for_inference = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

# Load LoRA adapters on top
trained_model = PeftModel.from_pretrained(
    base_model_for_inference,
    "./dpo-lora-checkpoints/final_model"
).to(device)

# Merge LoRA weights into base model (optional, for faster inference)
trained_model = trained_model.merge_and_unload()

print("✅ LoRA model loaded and merged!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

helpful-base/train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

helpful-online/train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

...

Tokenizing eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


✅ DPO Trainer with LoRA initialized


Step,Training Loss
10,0.689400
20,0.697600
30,0.692300
40,0.695500
50,0.722100
60,0.697500
70,0.699300
80,0.699500
90,0.701000
100,0.690800


✅ Training complete!
✅ LoRA adapters saved!
✅ LoRA model loaded and merged!


In [5]:
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained("./dpo-lora-checkpoints/final_model")
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained("gpt2").to(DEVICE)
trained_model = AutoModelForCausalLM.from_pretrained("./dpo-lora-checkpoints/final_model").to(DEVICE)

print("Models loaded for evaluation.")


Models loaded for evaluation.


In [6]:
def compute_kl(model_a, model_b, text):
    enc = tokenizer(text, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        logits_a = model_a(**enc).logits
        logits_b = model_b(**enc).logits

    p = F.log_softmax(logits_a, dim=-1)
    q = F.log_softmax(logits_b, dim=-1)

    return F.kl_div(p, q, log_target=True).item()

kl_values = []
for i in range(50):
    text = dataset["test"][i]["chosen"]
    kl = compute_kl(trained_model, base_model, text)
    kl_values.append(kl)

print("Avg KL Divergence:", sum(kl_values)/len(kl_values))


/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:3359: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Avg KL Divergence: 3.124018871858425e-06


In [7]:
import math
def compute_ppl(model, dataset, max_samples=200):
    model.eval()
    losses = []

    for i in tqdm(range(min(max_samples, len(dataset)))):
        row = dataset[i]
        text = row["chosen"]
        enc = tokenizer(text, return_tensors="pt").to(DEVICE)

        with torch.no_grad():
            out = model(**enc, labels=enc["input_ids"])
        losses.append(out.loss.item())

    return math.exp(sum(losses) / len(losses))

test_data = dataset["test"]

ppl_base = compute_ppl(base_model, test_data)
ppl_trained = compute_ppl(trained_model, test_data)

print("PPL (Base GPT-2):", ppl_base)
print("PPL (After DPO-LoRA):", ppl_trained)


  0%|          | 0/200 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


  0%|          | 0/200 [00:00<?, ?it/s]

PPL (Base GPT-2): 21.588911051821103
PPL (After DPO-LoRA): 23.182212848676723
